<font size="6">Tutorial: Unsupervised learning for molecular nonadiabatic dynamics</font> <br><br> <font size="5">Seminars on Machine Learning in Quantum Chemistry and Quantum Computing for Quantum Chemistry (SMLQC)</font> 

In this tutorial, we will see practical examples of using unsupervised machine learning methods **to automate pattern discovery and get chemical insight** from data generated by **nonadiabatic molecular dynamics (NAMD)** simulations. The NAMD data obtained within the **surface hopping** approximation is composed of an ensemble of trajectories that can be viewed as *multivariate time series* objects, where each point in time corresponds to a molecular geometry with its associated quantum properties. Thus, owing to the high dimensionality of the NAMD data, it can be cumbersome to identify the key internal coordinates of the molecule driving the excited-state dynamics by "manual" inspection of the data. This is the scenario where unsupervised learning comes to the rescue. The main idea is to use algorithms designed to find natural grouping structures within the data - **clustering analysis** - or find a compact data representation - **dimension reduction** - based on a given similarity measure between data instances.

To automate the unsupervised learning analysis in the context of nonadiabatic dynamics, we have developed a Python package called [**ULaMDyn**](www.ulamdyn.com), which provides a complete pipeline for analyzing NAMD trajectories data generated by the [**Newton-X**](https://newtonx.org/) program. This pipeline starts with collecting data from the Newton-X outputs, going through molecular representations, dimension reduction, and clustering analysis. Although ULaMDyn provides a friendly-user command-line interface to perform the whole data analysis in a single shot, in this tutorial, we will unfold the pipeline step-by-step to get a better understanding and more flexibility in the data analysis process. 

# Installation of required packages

**Instruction**: If you are running this notebook in Google colab, please restart the execution environment after the packages' installation given by the bash commands below. This is necessary to avoid error when instantiating the classes of ulamdyn.

In [ ]:
%%bash

git clone https://gitlab.com/maxjr82/ulamdyn.git
cd ulamdyn
pip install -e .
pip install py3Dmol

# The NAMD dataset: fulvene

The fulvene is used in this tutorial as an example of photoactive molecule that undergoes structural transformation in a nonadiabatic dynamics simulations starting from the first excited-state. To generate the fulvene dataset, the Newton-X program was used to propagate 200 surface hopping trajectories up to 60 fs with a time step of 0.1 fs. The CAS(6,6)/6-31G* method was used to compute the quantum chemical properties for the two electronic states (S$_0$ and S$_1$). For the sake of time and simplicity, only a fraction of the total number of trajectories (50 trajectories) was selected for the tutorial. The full dataset is available to download at https://doi.org/10.6084/m9.figshare.14446998.v1.

In [ ]:
%%bash

wget https://github.com/maxjr82/smlqc_ulamdyn/raw/main/nx_trajs_fulv.tgz
tar -zxvf nx_trajs_fulv.tgz
rm -rf nx_trajs_fulv.tgz

After downloading and unpacking the NAMD data, we need to move to the working directory that contains the 50 TRAJ folders with the simulation results. ULaMDyn will automatically recognize the available trajectories to extract all the information required to perform the analysis. 

In [ ]:
import os

TUTORIAL_DIR = "nx_trajs_fulv"

os.chdir(TUTORIAL_DIR)
os.listdir('./')

# ULaMDyn via command-line interface

The command-line interface (CLI) of ULaMDyn provides an alias to a set of predefined wrapper functions to assist through the complete process of performing unsupervised learning analysis on NAMD data. Alternatively, one can also use the CLI to easily extract the relevant computed quantities available in the multiple trajectories and export the collected information as structured datasets in csv format. To check for the options available in the CLI, one can run the help function in a shell terminal.  

In [ ]:
! run-ulamdyn --help

For a quick demonstration, let us use the command-line interface of ULaMDyn to run a complete clustering analysis with the K-Means method. In the example shown below, we provide the type of data in which the clustering analysis will be performed with the option `-space=geoms` (this means that geometries from different trajectories and time will be compared), then the number of geometries randomly selected from the full dataset is also given (`--n_samples=500`), and finally the descriptor in which the XYZ coordinates will be converted (`--descriptor=RE`).

In [ ]:
! run-ulamdyn clustering --space=geoms --n_samples=500 --descriptor=RE --method=kmeans --n_clusters=2 

In [ ]:
# Clean the working directory
%%bash

rm -rf kmeans* *.csv

# Step-by-step pipeline for unsupervised analysis

As shown in the CLI example above, ULaMDyn performed the complete clustering analysis in an automated way, starting from the data collection, then converting geometries into a descriptor, and finally running the clustering algorithm on the geometries' space. In addition, the program performed several statistical analysis by groupping the data according the cluster labels provided by the clustering algorithm. In the next, we will unfold this pipeline process to see how ULaMDyn can be used to perform this analysis step-by-step in a Python framework.

In [ ]:
import numpy as np
import pandas as pd
import ulamdyn as umd

# Packages for visualization
import py3Dmol
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# Plot settings

%matplotlib inline

mpl.rcParams['figure.figsize'] = [7.8,5.9]
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['xtick.labelsize'] = 13
mpl.rcParams['ytick.labelsize'] = 13
mpl.rcParams['legend.fontsize'] = 15

plt.style.use('seaborn-white')
sns.set(color_codes=True)

legend_settings = {'loc':'upper center', 'ncol':3, 'frameon':True, 'facecolor':'white', 
                   'framealpha':0.8, 'bbox_to_anchor':(0.5, 1.11)}

In [ ]:
def view_molecule(xyz_geom, style):
    
    for k in style.keys():
        assert k in ('line', 'stick', 'sphere', 'carton')
    
    molview = py3Dmol.view(width=350,height=350)
    molview.addModel(xyz_geom,'xyz')
        
    molview.setStyle(style)
    molview.setBackgroundColor('0xeeeeee')
    molview.zoomTo()
    
    return molview

# Read and inspect data

The first step in the pipeline for analyzing the nonadiabatic MD data is to collect the relevant quantities (e.g., molecular geometries, potential energy for each electronic state, kinetic energy, energy-gradients, oscillator strength, etc.) computed by the MD program for each trajectory. In the case of Newton-X CS, this information is typically outputted in unstructured text files that you can find in each `TRAJXX/RESULTS` folder. Thus, ULaMDyn provides built-in classes to collect all these data and stores them in a Python object for easy manipulation. In the next, we will see how these data collection classes can be used in a Python environment.

## Collect molecular geometries from NAMD trajectories

In [ ]:
geoms_loader = umd.GetCoords()
geoms_loader.read_all_trajs()

In [ ]:
print(geoms_loader)

Once all molecular geometries have been collected and loaded into the class variable, one can easily access any particular geometry of the dataset in the XYZ format by specifying the `TRAJ` and `time` indices. Then, the selected geometry can be visualized in the Jupyter notebook by using py3Dmol.

In [ ]:
# Select geometry either from trajectory and time indices
# or by the geometry number in the loaded dataset.
geometry = geoms_loader[9,10.0]
print(geometry)

In [ ]:
s = {'stick': {'radius': .15}, 'sphere': {'scale': 0.20}}
view_molecule(geometry, s)

If a reference geometry named `geom.xyz` is available in the working directory, the `read_all_trajs()` method will compute, by default, the RMSD between each current geometry read from the trajectories and a given reference geometry. In our example, the reference geometry is the ground-state one (S$_0$ minimum). For some unsupervised learning algorithms implemented in ULaMDyn, it is also possible to use the RMSD as a distance metric to compare pair of geometries.

In [ ]:
rmsd_vals = geoms_loader.rmsd
hist = plt.hist(rmsd_vals, bins=50, alpha=0.8)
plt.xlabel(r"RMSD $\AA$")
plt.ylabel("Frequency")
plt.show()

## Build dataset with chemical properties

The quantum mechanical quantities such as potential energies and oscillator strength are collected from the Newton-X outputs using the `GetProperties` class. In the case of fulvene dynamics, we have available the potential energy of the ground and first excited-state, the oscillator strength corresponding to the transition between these two states, and also the MCSCF coefficients of the CAS wave function. In addition, there is also a function to collect the state's population computed by Newton-X. In the next cells, we will see examples of how to extract these data.

In [ ]:
properties_loader = umd.GetProperties()

In [ ]:
df_props = properties_loader.energies()

In [ ]:
df_props

Note that, for consistency, the datasets generated by ULaMDyn will contain two primary indices, corresponding to the columns `TRAJ` and `time`, which allow to identify each data point in the whole set of NAMD trajectories available.

In [ ]:
_ = properties_loader.populations()

In [ ]:
df_props = properties_loader.dataset
df_props

In [ ]:
egap = df_props['DE21'].values
hist = plt.hist(egap, bins=50, alpha=0.8)
plt.xlabel(r"S$_0$-S$_1$ energy gap (eV)")
plt.ylabel("Frequency")
plt.show()

## Geometry-based descriptors

In [ ]:
geoms_loader.xyz

In ULaMDyn, there are two classes of **symmetry-aware descriptors** (translational and rotational invariant) based on molecular geometries: the **pairwise atom-atom distances** (R2 family of descriptors) and the **Z-Matrix** representation. Each one of these descriptors is handled by Python classes that take a `GetCoords()` object as input to access the NAMD molecular geometries and convert them into the specified descriptor type. As you will see in the example below, the R2 descriptor class contains the function `build_descriptor()`, which returns a Pandas data frame object with the descriptor calculated for all geometries of each NAMD trajectory. Other variants of the R2 descriptor supported by this function include:

- *inv-R2* -> inverse of the R2 matrix
- *delta-R2* -> difference in the R2 descriptor of each MD frame and a reference geometry
- *RE* -> inverse R2 of each MD frame normalized by the R2 vector of a reference geometry 

In [ ]:
atom_dist = umd.R2(geoms_loader)
df_r2 = atom_dist.build_descriptor(variant='R2')
df_r2.head()

In [ ]:
idx_all_hops = df_props.query("Hops_S21 == 1").index.tolist() + df_props.query("Hops_S12 == 1").index.tolist()
idx_no_hops = df_props.query("Hops_S21 == 0 and Hops_S12 == 0").index.tolist()

h1 = plt.hist(df_r2.iloc[idx_all_hops]['r56'], bins=20, color='red', alpha=0.5, label='Hops')
h2 = plt.hist(df_r2.iloc[idx_no_hops]['r56'].sample(len(idx_all_hops)), bins=20, color='blue', 
              alpha=0.5, label='No hops')
plt.xlabel(r"C5-C6 bond distance ($\AA$)")
plt.ylabel("Frequency")
plt.legend()
plt.show()

In [ ]:
for traj in df_r2['TRAJ'].unique():
  t = df_r2.query(f"TRAJ == {traj}")['time'].values
  c5_c6_dist = df_r2.query(f"TRAJ == {traj}")['r56'].values
  plt.plot(t, c5_c6_dist, c='k', lw=0.3, alpha=0.5)
plt.scatter(df_r2.iloc[idx_all_hops]['time'], df_r2.iloc[idx_all_hops]['r56'], 
            color='red', s=1.5)  
plt.xlabel("Time (fs)")
plt.ylabel(r"C5-C6 bond distance ($\AA$)")
plt.xlim(0,60)
plt.tight_layout()
plt.show() 

## Dimensionality reduction

In unsupervised learning analysis, dimensionality reduction is a key concept that refers to the process of reducing the number of features or variables in a dataset. **The main goal is to find a low-dimensional representation of the data while still capturing the most important information contained in the data**, thereby reducing the complexity of the analysis. This is important because as the number of features or variables increases, the difficulty of visualizing and analyzing the data also increases. By compressing the data in a meaningful way, it becomes easier to identify patterns, relationships, and clusters within the data. There are various techniques used in dimension reduction, including *principal component analysis* (PCA), *isometric feature mapping* (Isomap), and *t-distributed stochastic neighbor embedding* (t-SNE), each of which has its own strengths and weaknesses. These algorithms can be used either as a data exploration tool for visual inspection of the data structure via scatter plots or as a preprocessing step to provide effectively compact input data for supervised learning methods.

In [ ]:
dimred = umd.DimensionReduction(data=df_r2, dt=0.5, scaler='standard')

### Isomap

In the example below, we will use the [Isomap algorithm](https://www.science.org/doi/10.1126/science.290.5500.2319), which is a manifold learning technique used as a non-linear feature reduction method that aims at preserving the geodesic distances between data points in the low-dimensional space. The main assumption behind Isomap is that the data, even though it may be recorded in a very high dimensional space, inherently leaves in a low dimensional structure which can be encoded in terms of a manifold. In a nutshell, the Isomap algorithm works as follows:

1. Given some abstract data points $X_1,...,X_n$ and a distance function $d(x_i,x_j)$;
2. Build a $k$-nearest neighbor graph (using fixed radius or KNN algorithm) where the edges are weighted by the distances. These are local distances;
3. In the $k$NN graph, compute the shortest path distances between all pair of data points and store them in a matrix D. They correspond to the geodesic distances;
3. Then apply the classical multidimensional scaling (MDS) method using the distance matrix D as input to find the low-dimensional space embedding that preserves the geodesic distances.

In [ ]:
df_isomap = dimred.isomap(n_components=2)

In [ ]:
idx = df_isomap.index.tolist()
colors = df_props['DE21'].iloc[idx].values
plt.scatter(df_isomap['X1'], df_isomap['X2'], c=colors, cmap='jet', s=3.0, alpha=0.5)
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Isomap @ R2 descriptor')
cbar = plt.colorbar(pad=0.01)
cbar.set_label(r"S$_0$-S$_1$ energy gap (eV)")
plt.tight_layout()
plt.show()

Note that, in the two-dimensional representation of the R2 descriptor generated by Isomap, the molecular geometries characterized by a large energy gap (red dots) appear separated from those corresponding to a small energy gap (blue dots). This qualitative picture indicates that the geometries near the crossing seams should have distinguished features compared to the other geometries.

In [ ]:
sns.regplot(df_isomap['X1'], df_r2.iloc[idx]['r56'], scatter_kws={'s': 3.0},
            line_kws={'color':'red'})
plt.ylabel(r"C5-C6 bond distance ($\AA$)")
plt.show()

In nonlinear dimensionality reduction, finding the relationship between the embedded dimensions and the original features to determine which one is contributing the most to the clustering patterns is usually complicated. One alternative to get an intuition about those relationships is to plot each geometrical feature of the molecules against the embedded dimensions.

## Gradient-based descriptors

Because the type of analysis we are doing here is essentially a postprocessing step on the NAMD simulation data, we do not need to be restricted to looking only at the molecular geometries. In principle, any quantum chemical information available in the simulations can be used as a descriptor for unsupervised learning analysis. For example, the energy-gradient matrices of each potential energy surface carry valuable information on how fast the geometries can change during the dynamics. Hence, we will use here the difference between the energy-gradient matrices of the S$_1$ and S$_0$ states as an example of descriptor.

In [ ]:
grads_loader = umd.GetGradients()
grads_loader.build_dataframe()

In [ ]:
df_gdiff = grads_loader.datasets['S2'] - grads_loader.datasets['S1']
df_gdiff.insert(0, "TRAJ", df_props['TRAJ'].values)
df_gdiff.insert(1, "time", df_props['time'].values)
df_gdiff.head()

In [ ]:
dimred = umd.DimensionReduction(data=df_gdiff, dt=0.5, scaler='standard')
df_isomap = dimred.isomap(n_components=2, metric='euclidean')

In [ ]:
idx = df_isomap.index.tolist()
colors = df_props['DE21'].iloc[idx].values
plt.scatter(df_isomap['X1'], df_isomap['X2'], c=colors, cmap='jet', s=3.0, alpha=0.5)
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Isomap @ Gradients difference', fontsize=15)
cbar = plt.colorbar(pad=0.01)
cbar.set_label(r"S$_0$-S$_1$ energy gap (eV)")
plt.tight_layout()
plt.show()

Although the Isomap diagram derived from the gradient difference descriptors looks different from the one obtained with the R2 descriptor (geometry-based), one can observe that geometries with small and large energy gaps between the S$_0$ and S$_1$ states still appear as distinct groups in the plot.

## Clustering analysis

[Clustering](https://en.wikipedia.org/wiki/Cluster_analysis) is a subfield of unsupervised data analysis where the learning task consists of finding commonalities in a set of objects so that objects sharing a high similarity with respect to a given metric fall into the same group (called a cluster). In contrast, dissimilar objects are assigned to distinct groups. In principle, it is not required to perform dimensionality reduction prior to clustering analysis since the similarity metric used to compare pairs of data points can be computed in the original high-dimensional space of the data. However, datasets with a very large number of features may lead to the [*curse of dimensionality*](https://en.wikipedia.org/wiki/Clustering_high-dimensional_data) problem, which tends to degrade the performance of clustering algorithms. So, in this case, it might be recommended to reduce the dimension of the data before applying a clustering method.

There are many different algorithms to perform clustering analysis that differ essentially in the understanding of what constitutes a cluster and how to find them. The algorithms available in ULaMDyn for clustering analysis are:

+ K-Means clustering
+ Hierarchical agglomerative clustering
+ Spectral clustering (equivalent to kernel K-Means)

**Primary goal**: split the NAMD dataset into smaller subgroups of similar molecular geometries to facilitate the identification of the key active internal coordinates related to the photochemical process. 

In [ ]:
zmt = umd.ZMatrix(geoms_loader)
df_dzmt = zmt.build_descriptor(delta=True, apply_to_delta='sigmoid')

In [ ]:
clustering = umd.ClusterGeoms(data=df_dzmt, dt=0.5, scaler='standard')
df_kmeans = clustering.kmeans(n_clusters=3)

In [ ]:
df_cluster = pd.merge(df_props, df_kmeans, left_index=True, right_index=True)
df_cluster

In [ ]:
select_cols = ['time', 'DE21', 'Hops_S21', 'Hops_S12']
df_cluster.groupby(by=['kmeans_labels']).mean()[select_cols].reset_index()

In [ ]:
sns.displot(data=df_cluster, x="DE21", hue="kmeans_labels", palette="Set1")
plt.xlabel(r"S$_0$-S$_1$ energy gap (eV)")
plt.show()

Once the cluster labels have been determined, we can use the Isomap low-dimensional representation to visualize how the data points are distributed in clusters. This is helpful to check the effectiveness of the clustering algorithm in identifying groups of similar data points.

In [ ]:
dimred = umd.DimensionReduction(data=df_dzmt, dt=0.5, scaler='standard')
df_isomap = dimred.isomap(n_components=2, metric='euclidean')
df_isomap = pd.merge(df_isomap, df_kmeans, left_index=True, right_index=True)

In [ ]:
sns.scatterplot(data=df_isomap, x='X1', y='X2', hue='kmeans_labels', alpha=0.8)
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Isomap @ Delta Z-Matrix', fontsize=15)
plt.show()

In [ ]:
zmat = umd.ZMatrix(geoms_loader)
df_zmt = zmat.build_descriptor()
df_zmt = pd.merge(df_zmt, df_kmeans, left_index=True, right_index=True)
df_zmt.head()

In [ ]:
sns.displot(data=df_zmt, x="r65", hue="kmeans_labels", palette="Set1")
plt.xlabel(r"C5-C6 bond distance ($\AA$)")
plt.show()

In [ ]:
cluster_labels = list(df_zmt['kmeans_labels'].unique())

colors = ['blue', 'gold', 'red']
idx_hops = df_cluster[(df_cluster['Hops_S21'] == 1) | (df_cluster['Hops_S12'] == 1)].index.tolist()

for l,c in zip(cluster_labels, colors):
    zmt = df_zmt[df_zmt['kmeans_labels'] == l].abs()
    plt.scatter(zmt['r65'], zmt['d12654'], s=8, c=c, alpha=0.4, 
                label='cluster ' + str(l))
    df_zmt_hops = df_zmt.abs().loc[idx_hops]
    zmt = df_zmt_hops[df_zmt_hops['kmeans_labels'] == l]
    if zmt.shape[0] != 0:
        plt.scatter(zmt['r65'], zmt['d12654'], s=150, alpha=0.8, marker='*', 
                    edgecolors='k', c=c)

plt.ylabel('H$_11$-C$_6$-C$_5$-C$_4$ angle ($^\circ$)', labelpad=10)
plt.xlabel('C$_5$-C$_6$ bond length ($\AA$)', labelpad=10)
plt.legend(**legend_settings)
plt.show()

In [ ]:
traj, time = df_cluster.query("kmeans_labels == 1").sample(1)[['TRAJ', 'time']].values.flatten()
print(f"  Geometry for TRAJ{int(traj)} | time = {time} fs")
s = {'stick': {'radius': .15}, 'sphere': {'scale': 0.20}}
view_molecule(geoms_loader[traj, time], s)